# Import Libraries & Data

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
file_name = "Brand Performance Raw Data.xlsx"
sheets_dict = pd.read_excel(file_name, engine="openpyxl", sheet_name=None)

raw_data = sheets_dict['Raw data']
market_data = sheets_dict['Market data']
market_share_data = sheets_dict['Market share calculations']

### Feature Engineering

#### One-hot encoding

In [3]:
raw_data['Date'] = pd.to_datetime(raw_data['Date'])

In [4]:
categorical_columns = raw_data.select_dtypes(include=['object']).columns.tolist()
#categorical_columns = raw_data['Brand']
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(raw_data[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded dataframe with the original dataframe
df_encoded = pd.concat([raw_data, one_hot_df], axis=1)

# Drop the original categorical columns
df_encoded = df_encoded.drop(categorical_columns, axis=1)

df_encoded

,Date,Media Budget,Revenue,monthly contribution to Market Share,Media Budget to Revenue Ratio,Sales Growth,ROI,Return on ad Spend,Brand_LuxeGlow,Brand_NaturEssence,Brand_PureRadiance,Brand_TrendyChic
0,2020-01-01,330000,9736612,0.11,0.033893,0,-1.000000,29.504885,1.0,0.0,0.0,0.0
1,2020-01-01,307000,8815331,0.31,0.034826,0,-1.000000,28.714433,0.0,1.0,0.0,0.0
2,2020-01-01,296000,9801435,0.31,0.030200,0,-1.000000,33.112956,0.0,0.0,0.0,1.0
3,2020-01-01,373000,9681593,0.09,0.038527,0,-1.000000,25.956013,0.0,0.0,1.0,0.0
4,2020-01-01,476000,10203030,0.18,0.046653,0,-1.000000,21.434937,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
175,2022-12-01,226000,13515252,0.42,0.016722,-1143903,-6.061518,59.802000,1.0,0.0,0.0,0.0
176,2022-12-01,421000,16628992,0.05,0.025317,4125839,8.800093,39.498793,0.0,1.0,0.0,0.0
177,2022-12-01,276000,13785379,0.06,0.020021,701322,1.541022,49.947025,0.0,0.0,0.0,1.0
178,2022-12-01,331000,14035171,0.33,0.023584,-2256791,-7.818100,42.402329,0.0,0.0,1.0,0.0


### Normalisation

Reasons for normalisation versus standardization:
    
* Maintains the interpretability of the original values within the specified range
* No assumptions about the distribution is made

In [5]:
feature_cols = ['Media Budget']
target = ['Revenue']
X = df_encoded[feature_cols]
y = df_encoded[target]


In [6]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Fit min-max scaler on training data
norm = MinMaxScaler().fit(X_train)

# Transform the training data
X_train_norm = norm.transform(X_train)
 
# Use the same scaler to transform the testing set
X_test_norm = norm.transform(X_test)

In [8]:
X_train_norm_df = pd.DataFrame(X_train_norm)
X_test_norm_df = pd.DataFrame(X_test_norm)

# Assigning original feature names for ease of read
X_train_norm_df.columns = feature_cols
X_test_norm_df.columns = feature_cols

X_train_norm_df.describe()

,Media Budget
count,135.000000
mean,0.510327
std,0.284152
min,0.000000
25%,0.290102
50%,0.518771
75%,0.732082
max,1.000000


### Bi-variate Linear Regression

In [9]:
import statsmodels.api as sm

feature_cols = ['Media Budget']
target = ['Revenue']
X = df_encoded[feature_cols]
y = df_encoded[target]

#add constant to predictor variables
x = sm.add_constant(X)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.336
Method:                 Least Squares   F-statistic:                     91.50
Date:                Sat, 01 Feb 2025   Prob (F-statistic):           9.43e-18
Time:                        14:04:47   Log-Likelihood:                -2847.9
No. Observations:                 180   AIC:                             5700.
Df Residuals:                     178   BIC:                             5706.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.007e+06   5.71e+05     12.283   

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Define the predictor variables
X = df_encoded.drop(columns=['Revenue','Date', 'Media Budget to Revenue Ratio', 'ROI', 'Return on ad Spend']) #Remove columns that are calculated using revenue to avoid autocorrelation

# Add a constant to the model (intercept)
X = add_constant(X)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

                                Feature        VIF
0                                 const  25.881523
1                          Media Budget   1.274622
2  monthly contribution to Market Share   1.037094
3                          Sales Growth   1.224640
4                        Brand_LuxeGlow   1.636633
5                    Brand_NaturEssence   1.650043
6                    Brand_PureRadiance   1.630625
7                      Brand_TrendyChic   1.607964


Predictors with VIF values below 3 indicate low multicollinearity. There’s some correlation with other predictors, but it’s usually not severe. 

### Modelling

In [13]:
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
from sklearn.linear_model import LinearRegression

feature_cols = ['Media Budget',  'Brand_LuxeGlow', 'Brand_NaturEssence', 
                'Brand_PureRadiance', 'Brand_TrendyChic']
target = 'Revenue'

X = df_encoded[feature_cols]
y = df_encoded[target]

model = LinearRegression()
model.fit(X, y)

def objective(**allocations):
    """Maximize total revenue over 12 months using Bayesian Optimization, ensuring total budget usage."""
    total_revenue = 0
    
    total_allocated = sum(allocations.values())
    
    scaled_allocations = {month: allocation * (total_budget / total_allocated) for month, allocation in allocations.items()}
    
    total_spent = sum(scaled_allocations.values())
    assert np.isclose(total_spent, total_budget), f"Total spend {total_spent} doesn't match the total budget {total_budget}"

    last_row = df_encoded.iloc[-1].copy()
    for month in range(12):
        last_row['Media Budget'] = scaled_allocations[f"month_{month+1}"]
        
        X_new_df = pd.DataFrame([last_row], columns=feature_cols)
        
        predicted_revenue = model.predict(X_new_df)[0]
        total_revenue += predicted_revenue
        
        last_row['Revenue'] = predicted_revenue  

    return total_revenue

total_budget = 22500000  

pbounds = {f"month_{i+1}": (0, total_budget) for i in range(12)}

optimizer = BayesianOptimization(
    f=objective,  
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(init_points=5, n_iter=20)

optimal_allocations = optimizer.max["params"]
print("Optimal Monthly Advertising Allocation:", optimal_allocations)

scaled_allocations = {month: allocation * (total_budget / sum(optimal_allocations.values())) 
                      for month, allocation in optimal_allocations.items()}

total_spent = sum(scaled_allocations.values())
print(f"Total spent in optimization: {total_spent}")
if np.isclose(total_spent, total_budget):
    print(f"Budget is fully allocated.")
else:
    print(f"Warning: Total budget not fully used, {total_budget - total_spent} remaining.")



|   iter    |  target   |  month_1  | month_10  | month_11  | month_12  |  month_2  |  month_3  |  month_4  |  month_5  |  month_6  |  month_7  |  month_8  |  month_9  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 4.113e+08 | 8.427e+06 | 2.139e+07 | 1.647e+07 | 1.347e+07 | 3.51e+06  | 3.51e+06  | 1.307e+06 | 1.949e+07 | 1.353e+07 | 1.593e+07 | 4.632e+05 | 2.182e+07 |
| 2         | 4.113e+08 | 1.873e+07 | 4.778e+06 | 4.091e+06 | 4.127e+06 | 6.845e+06 | 1.181e+07 | 9.719e+06 | 6.553e+06 | 1.377e+07 | 3.139e+06 | 6.573e+06 | 8.243e+06 |
| 3         | 4.113e+08 | 1.026e+07 | 1.767e+07 | 4.493e+06 | 1.157e+07 | 1.333e+07 | 1.045e+06 | 1.367e+07 | 3.837e+06 | 1.464e+06 | 2.135e+07 | 2.173e+07 | 1.819e+07 |
| 4         | 4.113e+08 | 6.854e+06 | 2.198e+06 | 1.54e+07  | 9.903e+06 | 2.746e+06 | 1.114e+07 | 7.737e+05 | 2.046e+07 | 5.823e+06 | 1.491e+07 | 7.01

In [ ]:
# Create a DataFrame to store the results
allocations_and_revenues = []

scaled_allocations = {month: allocation * (total_budget / sum(optimal_allocations.values())) 
                      for month, allocation in optimal_allocations.items()}

total_revenue = 0
last_row = df_encoded.iloc[-1].copy()

for month in range(12):
    last_row['Media Budget'] = scaled_allocations[f"month_{month+1}"]
    X_new_df = pd.DataFrame([last_row], columns=feature_cols)
    predicted_revenue = model.predict(X_new_df)[0]
    total_revenue += predicted_revenue
    
    allocations_and_revenues.append({
        "Month": f"Month {month+1}",
        "Allocated Budget": scaled_allocations[f"month_{month+1}"],
        "Predicted Revenue": predicted_revenue
    })

df_allocations_and_revenues = pd.DataFrame(allocations_and_revenues)

df_allocations_and_revenues.to_csv('budget_allocations_and_revenue_predictions.csv', index=False)

pd.set_option('display.float_format', '{:.12f}'.format)
df_allocations_and_revenues

,Month,Allocated Budget,Predicted Revenue
0,Month 1,3476048.366306452081,57689047.196615338326
1,Month 2,2247601.017940437421,39726715.772171065211
2,Month 3,884857.084435643395,19800702.489776208997
3,Month 4,2463288.959005777258,42880500.333491399884
4,Month 5,3222311.629340181127,53978913.937743887305
5,Month 6,2617131.738204602152,45129986.232014045119
6,Month 7,2096953.558722949820,37523951.775558397174
7,Month 8,891753.925061208662,19901547.949097488075
8,Month 9,994409.623110792134,21402577.418066553771
9,Month 10,207627.259929497523,9898262.165835537016


In [19]:
print(df_allocations_and_revenues['Allocated Budget'].sum())
print(df_allocations_and_revenues['Predicted Revenue'].sum())

22500000.000000007
411342617.80736244


### Model Evaluation

In [20]:
# Categorize revenue into classes
def categorize_revenue(revenue):
    if revenue < 10000000:  
        return 'low'
    elif revenue < 20000000:  
        return 'medium'
    else:  
        return 'high'

predicted_classes = []
true_classes = []

optimal_allocations = optimizer.max["params"] 
scaled_allocations = {month: allocation * (total_budget / sum(optimal_allocations.values())) 
                      for month, allocation in optimal_allocations.items()}


last_row = df_encoded.iloc[-1].copy()  

for month in range(12):
    last_row['Media Budget'] = scaled_allocations[f"month_{month+1}"]
    
    X_new_df = pd.DataFrame([last_row], columns=feature_cols)
    
    predicted_revenue = model.predict(X_new_df)[0]
    
    predicted_class = categorize_revenue(predicted_revenue)
    predicted_classes.append(predicted_class)
    
    true_class = categorize_revenue(last_row['Revenue']) 
    true_classes.append(true_class)


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted', labels=['low', 'medium', 'high'])
recall = recall_score(true_classes, predicted_classes, average='weighted', labels=['low', 'medium', 'high'], zero_division=0)
f1 = f1_score(true_classes, predicted_classes, average='weighted', labels=['low', 'medium', 'high'])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.1667
Precision: 1.0000
Recall: 0.1667
F1 Score: 0.2857


Poor evaluation metrics are expected as the model does not have an informative dataset. 